In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np

In [7]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [8]:
# Load dataset
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# reshape the image with to single channel
train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# padding to change image size to 32x32
train_images = np.pad(train_images, ((0,0),(2,2),(2,2),(0,0)), 'constant')
test_images = np.pad(test_images, ((0,0),(2,2),(2,2),(0,0)), 'constant')

train_images.shape

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


(60000, 32, 32, 1)

In [9]:
train_images = train_images.astype('float32') / 255
test_images = test_images.astype('float32') / 255

# one hot encode labeling
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [10]:
def build_lenet5(input_shape=(32, 32, 1), num_classes=10):
  model = models.Sequential()

  # Conv1
  model.add(layers.Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=input_shape, name='C1'))
  # Pool1
  model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'))
  # Conv2
  model.add(layers.Conv2D(16, kernel_size=(5, 5), activation='tanh', name='C3'))
  # Pool2
  model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'))
  model.add(layers.Flatten(name='Flatten'))
  # FC1
  model.add(layers.Dense(120, activation='tanh', name='C5_FC1'))
  # FC2
  model.add(layers.Dense(84, activation='tanh', name='FC2'))
  # Output
  model.add(layers.Dense(num_classes, activation='softmax', name='Output'))

  return model

In [11]:
model = build_lenet5()
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ C1 (Conv2D)                     │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ S2 (AveragePooling2D)           │ (None, 14, 14, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C3 (Conv2D)                     │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ S4 (AveragePooling2D)           │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten (Flatten)               │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ C5_FC1 (Dense)                  │ (None, 120)            │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FC2 (Dense)                     │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

In [12]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# Train
history = model.fit(train_images, train_labels,
                    epochs=10,
                    batch_size=64,
                    validation_data=(test_images, test_labels))

Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.8578 - loss: 0.4872 - val_accuracy: 0.9622 - val_loss: 0.1235
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9665 - loss: 0.1087 - val_accuracy: 0.9743 - val_loss: 0.0828
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9795 - loss: 0.0661 - val_accuracy: 0.9803 - val_loss: 0.0641
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9840 - loss: 0.0497 - val_accuracy: 0.9825 - val_loss: 0.0540
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9880 - loss: 0.0377 - val_accuracy: 0.9828 - val_loss: 0.0502
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.9899 - loss: 0.0316 - val_accuracy: 0.9833 - val_loss: 0.0526
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.9930 - loss: 0.0219 - val_accuracy: 0.9838 - val_loss: 0.0516
Epoch 8/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9935 - loss: 0.0206 - val_accuracy: 0

In [14]:
# Eval
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('Test accuracy:', test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9840 - loss: 0.0546
Test accuracy: 0.98580002784729


In [15]:
# Trying Predict
sample_images = test_images[:10]
sample_labels = np.argmax(test_labels[:10], axis=1)

predictions = model.predict(sample_images)
predicted_labels = np.argmax(predictions, axis=1)

for i in range(len(sample_images)):
    print(f"Image {i+1}: True Label = {sample_labels[i]}, Predicted Label = {predicted_labels[i]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 543ms/step
Image 1: True Label = 7, Predicted Label = 7
Image 2: True Label = 2, Predicted Label = 2
Image 3: True Label = 1, Predicted Label = 1
Image 4: True Label = 0, Predicted Label = 0
Image 5: True Label = 4, Predicted Label = 4
Image 6: True Label = 1, Predicted Label = 1
Image 7: True Label = 4, Predicted Label = 4
Image 8: True Label = 9, Predicted Label = 9
Image 9: True Label = 5, Predicted Label = 5
Image 10: True Label = 9, Predicted Label = 9


In [17]:
# Work of Gemini

def export_weights_to_header(model, filename="lenet_weights.h"):
    """
    Extracts weights and biases from a Keras model and writes them
    to a C header file.
    """
    print(f"\nExporting weights to {filename}...")
    with open(filename, 'w') as f:
        f.write("#ifndef LENET_WEIGHTS_H\n")
        f.write("#define LENET_WEIGHTS_H\n\n")
        f.write("// This file was generated by an automated Python script.\n")
        f.write("// It contains the weights and biases for the LeNet-5 model.\n\n")

        for layer in model.layers:
            weights_and_biases = layer.get_weights()
            if not weights_and_biases:  # If the layer has no trainable parameters (e.g., Pooling)
                continue

            # Layer name for C variable names
            layer_name = layer.name.replace('-', '_')

            # There are two items: weights and biases
            weights = weights_and_biases[0]
            biases = weights_and_biases[1]

            # --- Write Weights ---
            f.write(f"// Layer: {layer.name} - Weights\n")
            # For Conv layers, weights are (height, width, in_channels, out_channels)
            # For Dense layers, weights are (input_features, output_features)
            # We will flatten them in C order (row-major).
            flat_weights = weights.flatten()
            f.write(f"const float {layer_name}_weights[] = {{\n    ")
            for i, w in enumerate(flat_weights):
                f.write(f"{w:.6f}f, ")
                if (i + 1) % 12 == 0:
                    f.write("\n    ")
            f.write("\n};\n\n")

            # --- Write Biases ---
            f.write(f"// Layer: {layer.name} - Biases\n")
            flat_biases = biases.flatten()
            f.write(f"const float {layer_name}_biases[] = {{\n    ")
            for i, b in enumerate(flat_biases):
                f.write(f"{b:.6f}f, ")
                if (i + 1) % 12 == 0:
                    f.write("\n    ")
            f.write("\n};\n\n")

        f.write("#endif // LENET_WEIGHTS_H\n")
    print(f"Successfully exported {len(model.layers)} layers to {filename}.")

In [18]:
export_weights_to_header(model)


Exporting weights to lenet_weights.h...
Successfully exported 8 layers to lenet_weights.h.
